## Part 1: Preliminary Code 

Here we've got a handy cell containing the pip installs of some of the more uncommon modules that you'll need to make this work.

In [1]:
!pip install folium
!pip install tabulate
!pip install ipywidgets

Here we import all of the modules and such that are needed to ensure that this notebook runs correctly. If you don't already have it installed, you will need Folium, ipywidgets, and tabulate. If you run the cell above this one it will automatically install all of these.

In [16]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from IPython.display import display, clear_output
from tabulate import tabulate
from folium.plugins import MarkerCluster

state_geo = r'./us-states.json' #reads in the location data for the state boundaries
state_data = pd.read_csv('50_us_states_all_data.csv') #reads in the climate data we collected for the different states (US territories/outlying regions not included)
marker_data= pd.read_fwf('marker_longlat.txt') #like pd.read_csv but for files with fixed width formatted lines, reads in marker locations (capitals!)
marker_data.columns= ['State', 'Latitude', "Longitude"] #adds in column names
state_data

,State,Avg_tempF,Avg_Rain_In,Avg_Snow_In,Sunny_Days,Morn_Humid,Aftnoon_Humid,Avg_Humid,ABR,Wtemp,Stemp,SpTemp,FallTemp
0,Alabama,62.8,58.3,1.6,99,58,52,55.0,AL,46.5,78.6,62.5,63.7
1,Alaska,26.6,22.5,74.5,61,41,64,52.5,AK,2.6,52.3,24.7,26.7
2,Arizona,60.3,13.6,0.3,193,85,25,55.0,AZ,43.6,78.1,58.2,61.3
3,Arkansas,60.4,50.6,5.2,123,61,49,55.0,AR,41.5,78.8,60.3,61.4
4,California,59.4,22.2,0.0,146,68,62,65.0,CA,46.2,73.4,57.1,60.9
5,Colorado,45.1,15.9,19.1,136,71,35,53.0,CO,25.8,65.2,43.7,45.9
6,Connecticut,49.0,50.3,40.5,82,56,52,54.0,CT,28.5,69.2,47.3,51.2
7,Delaware,55.3,45.7,20.2,97,79,54,66.5,DE,36.1,74.2,53.3,57.4
8,Florida,70.7,54.5,0.0,101,66,57,61.5,FL,59.4,81.0,69.9,72.7
9,Georgia,63.5,50.7,0.7,112,66,50,58.0,GA,47.8,78.7,63.1,64.6


Here we do some shenanigans with the marker data set. Ignore this for now- it'll come in handy later when we start displaying data. 

In [7]:
#masks the state data for just the climate categories (still in alphabetical order by state)

state= state_data['State']
temp_marker= state_data['Avg_tempF']
rain_marker=  state_data['Avg_Rain_In']
snow_marker=  state_data['Avg_Snow_In']
humid_marker=  state_data['Avg_Humid']
sunny_marker=  state_data['Sunny_Days']

#------------------------------------------------------------------------------------------

#masks the state dataset for the temperature data and other datas

temp_data = state_data[['State','ABR','Avg_tempF']] 
rain_data = state_data[['State','ABR','Avg_Rain_In']] 
snow_data = state_data[['State','ABR','Avg_Snow_In']] 
humi_data = state_data[['State','ABR','Avg_Humid']] 
sunn_data = state_data[['State','ABR','Sunny_Days']] 
fall_data= state_data[['State','ABR','FallTemp']]
summer_data= state_data[['State','ABR','Stemp']]
spring_data= state_data[['State','ABR','SpTemp']]
winter_data=state_data[['State','ABR','Wtemp']]

#------------------------------------------------------------------------------------------

#sorts the temperature data from least to greatest

temp_data_sorted = temp_data.sort_values('Avg_tempF') 
rain_data_sorted = rain_data.sort_values('Avg_Rain_In')
snow_data_sorted = snow_data.sort_values('Avg_Snow_In')
humi_data_sorted = humi_data.sort_values('Avg_Humid')
sunn_data_sorted = sunn_data.sort_values('Sunny_Days')


#------------------------------------------------------------------------------------------

temp_high = temp_data_sorted.iloc[:-4:-1]    #filters the data for the top three results
temp_low = temp_data_sorted.iloc[:3]      #filters the data for the lowest three results
rain_high = rain_data_sorted.iloc[:-4:1]
rain_low = rain_data_sorted.iloc[:3]
snow_high = snow_data_sorted.iloc[:-4:1]
snow_low = snow_data_sorted.iloc[:3]
humi_high = humi_data_sorted.iloc[:-4:1]
humi_low = humi_data_sorted.iloc[:3]
sunn_high = sunn_data_sorted.iloc[:-4:1]
sunn_low = sunn_data_sorted.iloc[:3]

The cell below does some little marker data stuff for later (latitude and longitudes for marker positions), initializes the blank folium map.

In [8]:
lats= marker_data.iloc[:, 1] #sets latitude data 
lons= marker_data.iloc[:, 2] #sets longitude data

locations = list(zip(lats, lons)) #creates a lost of where the markers should be placed

### Part 2: The Cool Stuff

Here we define a lot of the mapping stuff, and go into making the widget! Also, note that the command line code merely silences the automatic map output of the widget upon using the drop-down menu- it doesn't mess with the redefinition of the variable linked to the dropdown menu.

In [9]:
mydata = {'Temperature Data':('Average Temperature','Avg_tempF','OrRd'), #creates a dictionary that the ipywidgets module can work with
          'Rain Data':('Average Rain (inches)', 'Avg_Rain_In', 'BuPu'),
          'Snow Data':('Average Snow (inches)','Avg_Snow_In', 'PuBu'),
          'Humidity Data':('Average Humidity','Avg_Humid', 'YlOrRd'),
          'Sunshine Data':('Average Sun','Sunny_Days', 'YlOrBr')}

default = 'Temperature Data' #defaults the widget to choose temperature data as an input

w = widgets.Dropdown(               #creates a dropdown menu with the data keys from the dictionary above as options
    options=mydata.keys(),
    value = default,
    description='Data',)

Below is a formatting function we later use in our widget to help with the tabular stuff. It goes through and applies datasets depending on the value chosen in the dropdown menu, as well as some conditional map markers!

In [10]:
def high_low_print(w): 
    
    """Prints highs and lows as filtered by the previous cell for each data type"""
    
    if w == 'Temperature Data': #will be explained further below, when we introduce the widget
        high = 'The highest average temperatures'
        low = 'The lowest average temperatures'
        name_high = temp_high
        name_low = temp_low
        popups= ['{}, Average Temperature: {} &degF'.format(state, temp)  for state in state for temp in temp_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Rain Data':
        high = 'The highest average rainfall states'
        low = 'The lowest average rainfall states'
        name_high = rain_high
        name_low = rain_low
        popups= ['{}, Average Rain (in.): {}'.format(state, rain) for state in state for rain in rain_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Snow Data':
        high = 'The highest average snowfall states'
        low = 'The lowest average snowfall states'
        name_high = snow_high
        name_low = snow_low
        popups= ["{}, Average Snow (in.): {}". format(state, snow) for state in state for snow in snow_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Humidity Data':
        high = 'The highest average humidity states'
        low = 'The lowest average humidity states'
        name_high = humi_high
        name_low = humi_low
        popups= ["{}, Average humidity(%): {}". format(state, humid) for state in state for humid in humid_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Sunshine Data':
        high = 'The sunniest states'
        low = 'The least sunniest states'
        name_high = sunn_high
        name_low = sunn_low
        popups= ['{}, Average Sunny Days (per year): {}'.format(state, sun) for state in state for sun in sunny_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    else:
        return print("No! Bad user! Choose a valid category!")
    
    print(high)                            #header
    print(tabulate(name_high)) #pretty prints the high data
    print()                                  #for spacing
    print(low)                           #header
    print(tabulate(name_low)) #and pretty prints the low data

The cells below plot all of the data, clickable markers included, and print out the top and bottom three states for the extremity of the chosen category.

In [11]:
m = folium.Map(location= [48, -102],  tiles= 'Cartodb Positron', zoom_start= 3) #creates a folium map using whatever data option was chosen for the widget

m.choropleth(geo_data= state_geo, data= state_data,
             columns= ['ABR', mydata[w.value][1]], 
             key_on= 'feature.id',
             fill_color= mydata[w.value][2], fill_opacity= 0.7, line_opacity= 0.2,
             legend_name= mydata[w.value][0], smooth_factor= 0.1,highlight= True)

tooltip = 'Click me!' #sets the tooltip for each marker with this instruction (for interactivity!)

high_low_print(w.value)

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        clear_output()
        display(w)
        high_low_print(w.value)
        m = folium.Map(location= [48, -102], zoom_start= 3)
        m.choropleth(geo_data= state_geo, data= state_data,
             columns= ['ABR', mydata[w.value][1]],
             key_on= 'feature.id',
             fill_color= mydata[w.value][2], fill_opacity= 0.7, line_opacity= 0.2,
             legend_name= mydata[w.value][0], smooth_factor=  0.1,highlight= True)
        display(m)
    

C:\Users\Maegan\Anaconda3\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [15]:
display(w) #displays the widget as described above

w.observe(on_change) #calls the function on_change to the widget

display(m) #displays the updates map

Dropdown(description='Data', index=4, options=('Temperature Data', 'Rain Data', 'Snow Data', 'Humidity Data', …

In [13]:
import pandas as pd
state_data = pd.read_csv('50_us_states_all_data.csv')

In [14]:
#Input statements about how people like their weather in each season.
preference=["hot","cold","humid","meh", 'rain', 'snow', 'sun']
#Ask each person what their preference is.
g=input("What's your weather preference?:")
while g not in preference:
    g=input("What's your weather preference?:")
if g=="hot":
    print(temp_high)  #Prints top 3 hottest temperatures.
    g=input("Which of these state would you want to live in?:")
    if g not in list(temp_high.iloc[:,0]):
        while g not in list(sunn_data_sorted.iloc[:,0]):
            g=input("Which of these state would you want to live in?:")
            
if g=="humid":
    print(humi_high)  #Prints top 3 most humid temp temperatures.
    g=input("Which of these states would you want to live in?:")
    if g not in list(humi_high.iloc[:,0]):
        h=list(humi_data_sorted.iloc[30:40,].iloc[:,0]) #List the top 10 most humid states
        print(h)
        g=input("Which of these other states would you like to live in?")
    while g not in list(state_data.iloc[:,0]):   #If it's not any of these states, then....
        g=input("Choose whatever state you want!")
    
if g=="cold":
    print(temp_low)  #Prints top 3 coolest temperatures.
    g=input("Which of these states would you want to live in?:")
    if g not in list(temp_low.iloc[:,0]):
        h=list(temp_data_sorted.iloc[4:10,].iloc[:,0]) #List the next 10 coolest states
        print(h)
        g=input("Which of these other states would you like to live in?")
    while g not in list(state_data.iloc[:,0]):   #If it's not any of these states, then....
        g=input("Choose whatever state you want!")
if g=="meh":
    g=int(input("At what temperatures do you like your summer season?"))
    if g <= 52.3:
        print("You really love the cold then! Alaska is the best we have!")
    else:
        l1=list(state_data[state_data["Stemp"] > g].iloc[:,0])
        #print(l1)
        h=int(input("At what temperatures do you like your winter season?"))
        if h >= max(state_data["Wtemp"]):
            print("You should move to",state_data[state_data["Wtemp"]==max(state_data["Wtemp"])].iloc[0,0])
        else:
            l2=list(state_data[state_data["Stemp"] > h].iloc[:,0])
            #print(l2)
            i=int(input("At what temperatures do you like your fall season?"))
            if i >= max(state_data["FallTemp"]):
                print("You should move to ",state_data[state_data["FallTemp"]==max(state_data["FallTemp"])].iloc[0,0])
            elif i <=min(state_data["FallTemp"]):
                print("You should live in",state_data[state_data["FallTemp"]==min(state_data["FallTemp"])].iloc[0,0])
            else:
                l3=list(state_data[state_data["FallTemp"] >= i].iloc[:,0])
                #print(l3)
                j=int(input("How much precipitation do you like?:"))
                if j >= max(state_data["Avg_Rain_In"]):
                    print("Well you will/should love",state_data[state_data["Avg_Rain_In"]==max(state_data["Avg_Rain_In"])].iloc[0,0])
                elif j <= min(state_data["Avg_Rain_In"]):
                    print("Well you will/should love",state_data[state_data["Avg_Rain_In"]==min(state_data["Avg_Rain_In"])].iloc[0,0])
                else:
                    l4=list(state_data[state_data["Avg_Rain_In"] >= i].iloc[:,0])
                    #print(l4)
                    state_list=[]
                    for i in l4:
                        if i in l3:
                            state_list.append(i)
                            for k in l3:
                                if k in l2:
                                    state_list.append(k)
                                    for l in l2:
                                        if l in l1:
                                            state_list.append(l)
                    print(np.unique(state_list))
                    m=input("Pick from these states:")
                    while m not in state_list:
                        m=input("Pick from these states:")
                        


        
 #       f=input("Pick a state:")
#if f not in list(state_data[state_data["Stemp"] > g].iloc[:,0]):
    #g=input("Well where else do you want to live?:")
#while g not in list(state_data.iloc[:,0]): #If it's not any of these states, then....
 #   g=input("Choose whatever state you want!")

        
    
            
    

What's your weather preference?:humid
            State ABR  Avg_Humid
25        Montana  MT       52.0
1          Alaska  AK       52.5
30     New Mexico  NM       52.5
11          Idaho  ID       52.5
5        Colorado  CO       53.0
28  New Hampshire  NH       53.5
31       New York  NY       53.5
44        Vermont  VT       53.5
34           Ohio  OH       53.5
36         Oregon  OR       53.5
6     Connecticut  CT       54.0
41      Tennessee  TN       54.5
43           Utah  UT       54.5
46     Washington  WA       54.5
19       Maryland  MD       54.5
42          Texas  TX       55.0
33      N. Dakota  ND       55.0
3        Arkansas  AR       55.0
2         Arizona  AZ       55.0
0         Alabama  AL       55.0
27         Nevada  NV       55.5
49        Wyoming  WY       55.5
16       Kentucky  KY       55.5
48      Wisconsin  WI       56.0
37   Pennsylvania  PA       56.0
21       Michigan  MI       56.0
32    N. Carolina  NC       56.0
22      Minnesota  MN       56.5
39   

False

In [62]:
state_data[state_data["Wtemp"]==max(state_data["Wtemp"])].iloc[0,0]

'Hawaii'

In [66]:
state_data

,State,Avg_tempF,Avg_Rain_In,Avg_Snow_In,Sunny_Days,Morn_Humid,Aftnoon_Humid,Avg_Humid,ABR,Wtemp,Stemp,SpTemp,FallTemp
0,Alabama,62.8,58.3,1.6,99,58,52,55.0,AL,46.5,78.6,62.5,63.7
1,Alaska,26.6,22.5,74.5,61,41,64,52.5,AK,2.6,52.3,24.7,26.7
2,Arizona,60.3,13.6,0.3,193,85,25,55.0,AZ,43.6,78.1,58.2,61.3
3,Arkansas,60.4,50.6,5.2,123,61,49,55.0,AR,41.5,78.8,60.3,61.4
4,California,59.4,22.2,0.0,146,68,62,65.0,CA,46.2,73.4,57.1,60.9
5,Colorado,45.1,15.9,19.1,136,71,35,53.0,CO,25.8,65.2,43.7,45.9
6,Connecticut,49.0,50.3,40.5,82,56,52,54.0,CT,28.5,69.2,47.3,51.2
7,Delaware,55.3,45.7,20.2,97,79,54,66.5,DE,36.1,74.2,53.3,57.4
8,Florida,70.7,54.5,0.0,101,66,57,61.5,FL,59.4,81.0,69.9,72.7
9,Georgia,63.5,50.7,0.7,112,66,50,58.0,GA,47.8,78.7,63.1,64.6


In [40]:
#g=state_data["FallTemp"] < 60
g=state_data["FallTemp"] < 60
h=list(humi_data_sorted.iloc[0:10,].iloc[:,0])
"Alaska" in h
h

['Montana',
 'Alaska',
 'New Mexico',
 'Idaho',
 'Colorado',
 'New Hampshire',
 'New York',
 'Vermont',
 'Ohio',
 'Oregon']

['Florida', 'Hawaii', 'Louisiana']